In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**1-) Lendo os dados**

1.1) Tabela principal dos filmes

In [ ]:
movies_df = pd.read_csv("../input/the-movies-dataset/movies_metadata.csv")
movies_df.head()

In [ ]:
movies_df.info()

In [ ]:
from datetime import datetime

def convert_float(b):
    try:
        return float(b)
    except:
        return None
def convert_datetime(d):
    try:
        return datetime.strptime(d,"%Y-%m-%d")
    except:
        return None
movies_df['revenue_float'] = movies_df['revenue'].astype(float)
movies_df['vote_average_float'] = movies_df['vote_average'].astype(float)
movies_df['vote_count_float'] = movies_df['vote_count'].astype(float)
movies_df['popularity_float'] = movies_df['popularity'].apply(lambda p: convert_float(p))
movies_df['runtime_float'] = movies_df['runtime'].astype(float)
movies_df['budget_float'] = movies_df['budget'].apply(lambda b: convert_float(b))
movies_df['release_date_datetime'] = movies_df['release_date'].apply(lambda d: convert_datetime(d))
movies_df['adult_bool'] = movies_df['adult'].astype(bool)
movies_df.head()

In [ ]:
movies_df.popularity_float

**2-) Construção de novas tabelas**


2.1) Genero dos filmes

In [ ]:
import json 

def parse_str_json(string):
    try:
        string = string.replace("'",'"')
        lista = json.loads(string)
        return lista
    except:
        return []
    

In [ ]:
genres_df = pd.DataFrame({'id':[],'name':[],'id_filme':[],})

for i in range(len(movies_df)):
    lista = parse_str_json(movies_df.genres.iloc[i])
    for j in range(len(lista)):
        lista[j]['id_filme'] = movies_df.id.iloc[i]
    genres_df = genres_df.append(lista,ignore_index=True,sort=False)
genres_df.head()


In [ ]:
genres_df.to_csv("genres.csv")

2.1) Companhias que fazem os filmes

In [ ]:
movies_df.production_companies.iloc[0]

In [ ]:
companies_df = pd.DataFrame({'id':[],'name':[],'id_filme':[],})

for i in range(len(movies_df)):
    lista = parse_str_json(movies_df.production_companies.iloc[i])
    for j in range(len(lista)):
        lista[j]['id_filme'] = movies_df.id.iloc[i]
    companies_df = companies_df.append(lista,ignore_index=True,sort=False)
companies_df.head()

In [ ]:
companies_df.to_csv("companies.csv")

2.1) Paises que fazem os filmes

In [ ]:
paises_df = pd.DataFrame({'iso_3166_1':[],'name':[],'id_filme':[],})

for i in range(len(movies_df)):
    lista = parse_str_json(movies_df.production_countries.iloc[i])
    if(isinstance(lista,list)==False):
        lista=[]
    for j in range(len(lista)):
        lista[j]['id_filme'] = movies_df.id.iloc[i]
    paises_df = paises_df.append(lista,ignore_index=True,sort=False)
paises_df.head()

In [ ]:
paises_df.to_csv("paises.csv")

**3-) Analises**

3.1-) Linguagens originais

In [ ]:
fig,ax = plt.subplots(figsize=(16,9))
languages = movies_df.original_language.value_counts()
print(f'Quantidade total de linguagens originais: {len(languages.keys())}')
sns.barplot(x = languages.keys()[:10], y = languages.values[:10])

* Como esperado, os filmes com linguagens originais em ingles dominam o dataset.

3.2-) Orçamento

In [ ]:
budgets = movies_df.budget_float[(movies_df.budget_float>0)]
budgets

In [ ]:
fig,ax = plt.subplots(figsize=(16,9))
sns.boxplot(data=budgets)

In [ ]:
fig,ax = plt.subplots(figsize=(16,9))
sns.histplot(data=budgets/(10**6),kde=True)
ax.set_xlabel("Valor do orçamento (MM $)")
ax.set_ylabel("Quantidade de filmes")
print(f'Quantidade de filmes com orçamento abaixo de 100 dolares: {budgets[budgets<100].sum()}')

* Grande quantidade de filmes sem orçamento, apenas 8890 apresentam, de um dataset contendo aproximadamente 45 mil
* Grande quantidade de outliers com valores altos, explicado pela maior parte dos filmes apresentarem orçamento baixo
    * Acredito que os outliers não possam ser retirados, mas sim analisados a grande quantidade de filmes com orçamento baixo

3.3-) Tempo do filme

In [ ]:
NaN_runtime = movies_df.runtime_float.isnull()

runtime_movies = movies_df.runtime_float[((movies_df.runtime_float>0) & (NaN_runtime==False)) ]
runtime_movies.sort_values()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,9))
sns.histplot(ax=ax[1],data=runtime_movies,kde=True)
ax[1].set_xlabel("Tempo do filme(min)")
ax[1].set_ylabel("Quantidade de filmes")

sns.boxplot(ax=ax[0],data=runtime_movies)


* Caso contrario ao do orçamento
    * Possivel retirada dos outliers altos
    * Analise para retirada dos outliers baixos
* Distribuição proxima a curva normal


3.4-) Avaliações

In [ ]:
NaN_ava = movies_df.vote_average_float.isnull()

avaliations_movies = movies_df.vote_average_float[((movies_df.vote_average_float>0) & (NaN_ava==False)) ]


count_ava_movies = movies_df.vote_count_float[((movies_df.vote_count_float>0) & (movies_df.vote_count_float.isnull()==False)) ]
count_ava_movies

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(16,9))

sns.boxplot(ax=ax[0,0],data=avaliations_movies)
sns.histplot(ax=ax[0,1],data=avaliations_movies,kde=True)
ax[0,1].set_xlabel("Nota")
ax[0,1].set_ylabel("Quantidade de filmes")

sns.boxplot(ax=ax[1,0],data=count_ava_movies)
sns.histplot(ax=ax[1,1],data=count_ava_movies,kde=True)
ax[1,1].set_xlabel("Quantidade de votos")
ax[1,1].set_ylabel("Quantidade de filmes")

1. Media de votos
* Dados bem consistentes
* Obedecem distribuição normal
* Outliers podem ser retirados, mas mecere uma analise antes

2. Quantidade de votos
* Inconstancia na distribuição e outliers - Sugestão: Adicionar essa info na media da nota, como 'força' estatistica da media

3.5-) Bilheteria

In [ ]:
NaN_bilhe = movies_df.revenue_float.isnull()

bilheteria_movies = movies_df.revenue_float[((movies_df.revenue_float>0) & (NaN_bilhe==False)) ]
bilheteria_movies

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,9))

sns.boxplot(ax=ax[0],data=bilheteria_movies)
sns.histplot(ax=ax[1],data=bilheteria_movies,kde=True)
ax[1].set_xlabel("Bilheteria")
ax[1].set_ylabel("Quantidade de filmes")

* Analise semelhante ao budget

3.6-) Correlação linear entre as variaveis 

In [ ]:
columns = ['revenue_float','vote_average_float','vote_count_float','popularity_float','runtime_float','budget_float']
sns.pairplot(data = movies_df[columns])

* As 6 variaveis analisadas não apresentam correlação linear entre elas

3.7-) Generos

In [ ]:
genres_df = pd.read_csv('./genres.csv').iloc[:,1:]
genres_df['id_filme'] = genres_df['id_filme'].astype(object)
genres_df.head() 

In [ ]:
genres_df.info() 

In [ ]:
genres_movies_df = genres_df.merge(right=movies_df,how='left',left_on='id_filme',right_on='id')
genres_movies_df.head()

In [ ]:
avaliacao_genero = genres_movies_df.groupby('name').vote_average_float.mean().sort_values(ascending=False)
bilheteria_genero = genres_movies_df.groupby('name').revenue_float.mean().sort_values(ascending=False)
bilheteria_genero

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(21,16))

sns.barplot(ax=ax[0,0],x = avaliacao_genero.keys()[:10], y = avaliacao_genero.values[:10])
sns.boxplot(ax=ax[1,0],data = avaliacao_genero)

ax[0,0].set_ylabel('Avaliação Média',fontsize=16)
ax[0,0].set_xlabel('Genero do filme',fontsize=16)
ax[1,0].set_ylabel('Avaliação Média',fontsize=16)

sns.barplot(ax=ax[0,1],x = bilheteria_genero.keys()[:10], y = bilheteria_genero.values[:10])
sns.boxplot(ax=ax[1,1],data =bilheteria_genero)

ax[0,1].set_ylabel('Bilheteria Média',fontsize=16)
ax[0,1].set_xlabel('Genero do filme',fontsize=16)
ax[1,1].set_ylabel('Bilheteria Média',fontsize=16)





* Alta mudança na bilheteria dos filmes após o genero de Ação
* Boxplot da bilheteria mostra que os demais generos apresentam valores ainda menores
* Pequena variação na avaliação na avaliação dos filmes extratificada por genero, verificada pelo boxplot
    * As 10 maiores avaliações medias não são tão maiores que as demais

3.8-) Produtoras dos filmes

In [ ]:
companies_df = pd.read_csv('./companies.csv').iloc[:,1:]
companies_df['id_filme'] = companies_df['id_filme'].astype(str)
companies_df.head() 

In [ ]:
companies_df.info() 

In [ ]:
companies_movies_df = companies_df.merge(right=movies_df,how='left',left_on='id_filme',right_on='id')
companies_movies_df.sample(10)

In [ ]:
avaliacao_companies = companies_movies_df.groupby('name').vote_average_float.mean().sort_values(ascending=False)
bilheteria_copanies = companies_movies_df.groupby('name').revenue_float.mean().sort_values(ascending=False)
bilheteria_copanies.keys()

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(21,16))

sns.barplot(ax=ax[0,0],x = avaliacao_companies.keys()[:5], y = avaliacao_companies.values[:5])
sns.boxplot(ax=ax[1,0],data = avaliacao_companies)

ax[0,0].set_ylabel('Avaliação Média',fontsize=16)
ax[0,0].set_xlabel('Produtora do filme',fontsize=16)
ax[1,0].set_ylabel('Avaliação Média',fontsize=16)

sns.barplot(ax=ax[0,1],x = bilheteria_copanies.keys()[:5], y = bilheteria_copanies.values[:5])
sns.boxplot(ax=ax[1,1],data =bilheteria_copanies)

ax[0,1].set_ylabel('Bilheteria Média',fontsize=16)
ax[0,1].set_xlabel('Produtora do filme',fontsize=16)
ax[1,1].set_ylabel('Bilheteria Média',fontsize=16)



3.9-) Paises que produzem os filmes

In [ ]:
paises_moveis = movies_df.merge(right=paises_df,how='left',left_on='id',right_on='id_filme')
paises_moveis.head()

In [ ]:
qtd_paises = paises_moveis.groupby('iso_3166_1').iso_3166_1.count().sort_values(ascending=False)[:20]



fig,ax = plt.subplots(figsize=(16,9))

sns.barplot(x = qtd_paises.keys(), y = qtd_paises.values,palette='pastel')
